In [ ]:
!wget https://lodmedia.hb.bizmrg.com/case_files/852825/train_dataset_train.csv
!wget https://lodmedia.hb.bizmrg.com/case_files/852825/test_dataset_test.csv

In [ ]:
!pip install catboost

In [ ]:
import time
import os
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import dbscan,DBSCAN
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier, SGDRegressor, RidgeClassifier, LogisticRegressionCV,Ridge,QuantileRegressor,PassiveAggressiveClassifier,PoissonRegressor
from sklearn.ensemble import ExtraTreesRegressor,ExtraTreesClassifier,RandomForestClassifier,VotingClassifier,RandomForestRegressor,GradientBoostingClassifier,GradientBoostingRegressor,StackingRegressor,BaggingClassifier
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.svm import LinearSVC,LinearSVR,SVR,NuSVR
from sklearn.decomposition import TruncatedSVD,PCA,FactorAnalysis,IncrementalPCA,FastICA,KernelPCA,NMF
from sklearn.preprocessing import RobustScaler,QuantileTransformer,PowerTransformer,PolynomialFeatures,KBinsDiscretizer,StandardScaler,OneHotEncoder,OrdinalEncoder,FunctionTransformer,MaxAbsScaler,MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline,FeatureUnion,TransformerMixin
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor,LocalOutlierFactor
from sklearn.model_selection import train_test_split,ShuffleSplit,StratifiedShuffleSplit,TimeSeriesSplit,GridSearchCV,KFold
from sklearn.feature_selection import SelectFromModel
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer,SimpleImputer
from sklearn.dummy import DummyRegressor,DummyClassifier
from sklearn import set_config
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error,roc_auc_score,accuracy_score,f1_score,classification_report,recall_score,make_scorer
import catboost

In [ ]:
train=pd.read_csv('/content/drive/MyDrive/NSK/train_dataset_train.csv')
test=pd.read_csv('/content/drive/MyDrive/NSK/test_dataset_test.csv')

In [ ]:
for cnt in range(2):
  train['Easting_Northing_'+str(cnt)]=PCA(2,random_state=0).fit_transform(train[['Easting', 'Northing']])[:,cnt]
train['Easting/Northing']=(MinMaxScaler(feature_range=(1, 100)).fit_transform(train[['Easting']])/
                             MinMaxScaler(feature_range=(1, 100)).fit_transform(train[['Northing']]))
train['Height/Reflectance']=(MinMaxScaler(feature_range=(1, 100)).fit_transform(train[['Height']])/
                             MinMaxScaler(feature_range=(1, 100)).fit_transform(train[['Reflectance']]))
train['Easting_Northing_0/Reflectance']=(MinMaxScaler(feature_range=(1, 100)).fit_transform(train[['Easting_Northing_0']])/
                             MinMaxScaler(feature_range=(1, 100)).fit_transform(train[['Reflectance']]))
train['Easting_Northing_1/Reflectance']=(MinMaxScaler(feature_range=(1, 100)).fit_transform(train[['Easting_Northing_1']])/
                             MinMaxScaler(feature_range=(1, 100)).fit_transform(train[['Reflectance']]))
train['Easting_Northing_0/Height']=(MinMaxScaler(feature_range=(1, 100)).fit_transform(train[['Easting_Northing_0']])/
                             MinMaxScaler(feature_range=(1, 100)).fit_transform(train[['Height']]))
train['Easting_Northing_1/Height']=(MinMaxScaler(feature_range=(1, 100)).fit_transform(train[['Easting_Northing_1']])/
                             MinMaxScaler(feature_range=(1, 100)).fit_transform(train[['Height']]))
train['Easting_Northing_0/Easting_Northing_1']=(MinMaxScaler(feature_range=(1, 100)).fit_transform(train[['Easting_Northing_0']])/
                             MinMaxScaler(feature_range=(1, 100)).fit_transform(train[['Easting_Northing_1']]))
train['Angle1']=np.arctan(train['Easting_Northing_0']/(60/np.tan(np.pi/6)))
train['Angle2']=np.arctan(train['Easting_Northing_1']/(60/np.tan(np.pi/6)))
train['Angle3']=np.arctan(np.sqrt(np.square(train['Easting_Northing_0'])+np.square(train['Easting_Northing_1']))/(60/np.tan(np.pi/6)))

In [ ]:
for cnt in range(2):
  test['Easting_Northing_'+str(cnt)]=PCA(2,random_state=0).fit(train[['Easting', 'Northing']]).transform(test[['Easting', 'Northing']])[:,cnt]
test['Easting/Northing']=(MinMaxScaler(feature_range=(1, 100)).fit(train[['Easting']]).transform(test[['Easting']])/
                             MinMaxScaler(feature_range=(1, 100)).fit(train[['Northing']]).transform(test[['Northing']]))
test['Height/Reflectance']=(MinMaxScaler(feature_range=(1, 100)).fit(train[['Height']]).transform(test[['Height']])/
                             MinMaxScaler(feature_range=(1, 100)).fit(train[['Reflectance']]).transform(test[['Reflectance']]))
test['Easting_Northing_0/Reflectance']=(MinMaxScaler(feature_range=(1, 100)).fit(train[['Easting_Northing_0']]).transform(test[['Easting_Northing_0']])/
                             MinMaxScaler(feature_range=(1, 100)).fit(train[['Reflectance']]).transform(test[['Reflectance']]))
test['Easting_Northing_1/Reflectance']=(MinMaxScaler(feature_range=(1, 100)).fit(train[['Easting_Northing_1']]).transform(test[['Easting_Northing_1']])/
                             MinMaxScaler(feature_range=(1, 100)).fit(train[['Reflectance']]).transform(test[['Reflectance']]))
test['Easting_Northing_0/Height']=(MinMaxScaler(feature_range=(1, 100)).fit(train[['Easting_Northing_0']]).transform(test[['Easting_Northing_0']])/
                             MinMaxScaler(feature_range=(1, 100)).fit(train[['Height']]).transform(test[['Height']]))
test['Easting_Northing_1/Height']=(MinMaxScaler(feature_range=(1, 100)).fit(train[['Easting_Northing_1']]).transform(test[['Easting_Northing_1']])/
                             MinMaxScaler(feature_range=(1, 100)).fit(train[['Height']]).transform(test[['Height']]))
test['Easting_Northing_0/Easting_Northing_1']=(MinMaxScaler(feature_range=(1, 100)).fit(train[['Easting_Northing_0']]).transform(test[['Easting_Northing_0']])/
                             MinMaxScaler(feature_range=(1, 100)).fit(train[['Easting_Northing_1']]).transform(test[['Easting_Northing_1']]))  
test['Angle1']=np.arctan(test['Easting_Northing_0']/(60/np.tan(np.pi/6)))
test['Angle2']=np.arctan(test['Easting_Northing_1']/(60/np.tan(np.pi/6)))
test['Angle3']=np.arctan(np.sqrt(np.square(test['Easting_Northing_0'])+np.square(test['Easting_Northing_1']))/(60/np.tan(np.pi/6)))

In [ ]:
X_train,X_val,y_train,y_val=train_test_split(train[['Easting_Northing_0', 'Easting_Northing_1', 'Height', 'Reflectance',
       'Height/Reflectance', 'Easting_Northing_0/Reflectance','Easting_Northing_1/Reflectance', 
       'Easting_Northing_0/Height','Easting_Northing_1/Height', 'Easting_Northing_0/Easting_Northing_1',
       'Angle1','Angle2','Angle3']],
                                             train['Class'],test_size=0.02,random_state=0,
                                             stratify=train['Class'])

In [ ]:
cls=catboost.CatBoostClassifier(iterations=2500, 
                                learning_rate=0.1,
                                loss_function='MultiClassOneVsAll',
                                use_best_model=True, 
                                auto_class_weights='Balanced', 
                                eval_metric='TotalF1:average=Macro')

In [ ]:
cls.fit(X_train,y_train,eval_set=(X_val,y_val),verbose=10)

In [ ]:
test['Class']=cls.predict(test[['Easting_Northing_0', 'Easting_Northing_1', 'Height', 'Reflectance',
       'Height/Reflectance', 'Easting_Northing_0/Reflectance','Easting_Northing_1/Reflectance', 
       'Easting_Northing_0/Height','Easting_Northing_1/Height', 'Easting_Northing_0/Easting_Northing_1',
       'Angle1','Angle2','Angle3']]).flatten()

In [ ]:
submit=test[['id','Class']]
submit.to_csv('submit.csv',index=False)